# Data anonymization with Microsoft Presidio(데이터 익명화)
##### 데이터 익명화는 개인 정보를 보호하고 기밀성을 유지하는 데 도움이 되기 때문에 언어 모델에 정보를 전달하기 전에 중요하다.
##### 개인 식별 정보(PII)를 모호하게 하거나 제거함으로 데이터 익명화를 유지한다.

##### 익명화는 두 단계로 구성된다.


*   식별 : 개인 식별 정보(PII)가 포함된 모든 데이터 필드를 식별한다.
*   대체 : 모든 PII를 정보를 참조용으로 사용할 수 있는 의사 값 또는 코드로 대체한다.



## 빠른 시작
##### 샘플 문장을 사용하여 PII 익명화 예제를 살펴보자

```python
# model download
!python -m spacy download en_core_web_lg

from langchain_experimental.data_anonymizer import PresidioAnonymizer

anonymizer = PresidioAnonymizer()

anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com"
)

# 이름, 전화번호, 이메일 익명화 수행
'My name is James Martinez, call me at (576)928-1972x679 or email me at lisa44@example.com'
```

### LCEL 표현
##### LCEL을 사용하면 나머지 애플리케이션과 익명화를 쉽게 연결할 수 있다.

```python
text = """Slim Shady recently lost his wallet.
Inside is some cash and his credit card with the number 4916 0387 9536 0861.
If you would find it, please call at 313-666-7440 or write an email here: real.slim.shady@gmail.com."""

from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI

anonymizer = PresidioAnonymizer()

template = """Rewrite this text into an official, short email:

{anonymized_text}"""
prompt = PromptTemplate.from_template(template)
llm = ChatOpenAI(temperature=0)

chain = {"anonymized_text": anonymizer.anonymize} | prompt | llm
response = chain.invoke(text)
print(response.content)

# PII 익명화 수행
Dear Sir/Madam,

We regret to inform you that Mr. Dennis Cooper has recently misplaced his wallet. The wallet contains a sum of cash and his credit card, bearing the number 3588895295514977.

Should you happen to come across the aforementioned wallet, kindly contact us immediately at (428)451-3494x4110 or send an email to perryluke@example.com.

Your prompt assistance in this matter would be greatly appreciated.

Yours faithfully,

[Your Name]
```

## 맞춤 익명화
##### analyzed_fields를 사용하여 특정 유형의 데이터만 익명화하도록 지정할 수 있다.

```python
anonymizer = PresidioAnonymizer(analyzed_fields=["PERSON"])

anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com"
)

# analyzed_fields로 지정한 사람 이름만 익명화
'My name is Shannon Steele, call me at 313-666-7440 or email me at real.slim.shady@gmail.com'
```

```python
anonymizer = PresidioAnonymizer(analyzed_fields=["PERSON", "PHONE_NUMBER"])
anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com"
)

# 이름과 번호 익명화
'My name is Wesley Flores, call me at (498)576-9526 or email me at real.slim.shady@gmail.com'
```

##### analyze_fields가 지정되지 않으면 기본적으로 익명 처리 장치는 지원되는 모든 형식을 감지한다

```python
['PERSON', 'EMAIL_ADDRESS', 'PHONE_NUMBER', 'IBAN_CODE', 'CREDIT_CARD', 'CRYPTO', 'IP_ADDRESS', 'LOCATION', 'DATE_TIME', 'NRP', 'MEDICAL_LICENSE', 'URL', 'US_BANK_NUMBER', 'US_DRIVER_LICENSE', 'US_ITIN', 'US_PASSPORT', 'US_SSN']
```

##### 탐지할 개인 데이터를 신중하게 정의해야한다!
##### 예를 들어, PHONE_NUMBER 필드는 폴란드어 전화번호를 지원하지 않으므로 다른 필드와 혼동한다.
##### 자신만의 인식기를 작성하여 존재하는 인식자 풀에 추가할 수 있다.

```python
# Pattern 객체 생성
from presidio_analyzer import Pattern, PatternRecognizer

polish_phone_numbers_pattern = Pattern(
    name="polish_phone_numbers_pattern",
    regex="(?<!\w)(\(?(\+|00)?48\)?)?[ -]?\d{3}[ -]?\d{3}[ -]?\d{3}(?!\w)",
    score=1,
)

# 하나 또는 더 많은 인식자를 정의한다.
polish_phone_numbers_recognizer = PatternRecognizer(
    supported_entity="POLISH_PHONE_NUMBER", patterns=[polish_phone_numbers_pattern]
)
```

```python
# 인식기 추가
anonymizer.add_recognizer(polish_phone_numbers_recognizer)
```

```python
print(anonymizer.anonymize("My polish phone number is 666555444"))
print(anonymizer.anonymize("My polish phone number is 666 555 444"))
print(anonymizer.anonymize("My polish phone number is +48 666 555 444"))

My polish phone number is <POLISH_PHONE_NUMBER>
My polish phone number is <POLISH_PHONE_NUMBER>
My polish phone number is <POLISH_PHONE_NUMBER>
```

##### 문제는 폴란드 전화번호를 인식하더라도 주어진 필드를 대체하는 방법을 알려주는 메서드가 없다. 아래와 같이 올바르게 교체해보자

```python
from faker import Faker

fake = Faker(locale="pl_PL")


def fake_polish_phone_number(_=None):
    return fake.phone_number()

fake_polish_phone_number()

'665 631 080'
```

```python
from presidio_anonymizer.entities import OperatorConfig

new_operators = {
    "POLISH_PHONE_NUMBER": OperatorConfig(
        "custom", {"lambda": fake_polish_phone_number}
    )
}

anonymizer.add_operators(new_operators)
anonymizer.anonymize("My polish phone number is 666555444")

'My polish phone number is 538 521 657'
```

## 중요한 고려사항
### 익명처리
##### 다양한 소스와 다양한 언어의 텍스트는 다양한 특성을 가지므로 감지 정밀도를 테스트하고 인식기와 연산자를 반복적으로 추가해라

### 인스턴스 익명화
##### PresidioAnonymizer의 경우 내장 메모리가 없으므로 같은 두 텍스트의 다른 결과를 나타낸다.

```python
print(anonymizer.anonymize("My name is John Doe. Hi John Doe!"))
print(anonymizer.anonymize("My name is John Doe. Hi John Doe!"))

My name is Robert Morales. Hi Robert Morales!
My name is Kelly Mccoy. Hi Kelly Mccoy!
```

##### 이전 익명화 결과를 보존하려면 내장 메모리가 있는 PresidioReversibleAnonymizer를 사용해라

```python
from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer

anonymizer_with_memory = PresidioReversibleAnonymizer()

print(anonymizer_with_memory.anonymize("My name is John Doe. Hi John Doe!"))
print(anonymizer_with_memory.anonymize("My name is John Doe. Hi John Doe!"))

My name is Ashley Cervantes. Hi Ashley Cervantes!
My name is Ashley Cervantes. Hi Ashley Cervantes!
```

# Reversible data anonymization with Microsoft Presidio(복원 가능한 데이터 익명화)
##### 복원 가능한 데이터 익명화는 중요한 PII를 마스킹하는 작업도 포함되지만, 권한이 있는 사용자가 필요할 때 이를 되돌릴 수도 있고 원본 데이터를 복원할 수 있다.



*   익명화 : PresidioAnonymizer와 마찬가지로 객체 자체는 구성된 값을 원본 값에 대한 매핑을 저장한다.
*   비익명화 : 매핑을 사용하여 가짜 데이터를 원본 데이터와 일치시킨 후 대체한다.

```python
    {
        "PERSON": {
            "<anonymized>": "<original>",
            "John Doe": "Slim Shady"
        },
        "PHONE_NUMBER": {
            "111-111-1111": "555-555-5555"
        }
        ...
    }
```



## 빠른 시작
##### PresidioReversibleAnonymizer는 익명화 측면에서 이전 버전(PresidioAnonymizer)과 크게 다르지 않다.

```python
from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer

anonymizer = PresidioReversibleAnonymizer(
    analyzed_fields=["PERSON", "PHONE_NUMBER", "EMAIL_ADDRESS", "CREDIT_CARD"],
    faker_seed=42,
)

anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com. "
    "By the way, my card number is: 4916 0387 9536 0861"
)

'My name is Maria Lynch, call me at 7344131647 or email me at jamesmichael@example.com. By the way, my card number is: 4838637940262'
```

##### 비익명화(deanonymize) 방법을 사용하여 프로세스를 되돌릴 수 있다.

```python
fake_name = "Maria Lynch"
fake_phone = "7344131647"
fake_email = "jamesmichael@example.com"
fake_credit_card = "4838637940262"

anonymized_text = f"""{fake_name} recently lost his wallet.
Inside is some cash and his credit card with the number {fake_credit_card}.
If you would find it, please call at {fake_phone} or write an email here: {fake_email}.
{fake_name} would be very grateful!"""

print(anonymizer.deanonymize(anonymized_text))

Slim Shady recently lost his wallet.
Inside is some cash and his credit card with the number 4916 0387 9536 0861.
If you would find it, please call at 313-666-7440 or write an email here: real.slim.shady@gmail.com.
Slim Shady would be very grateful!
```

### LCEL 표현

```python
text = """Slim Shady recently lost his wallet.
Inside is some cash and his credit card with the number 4916 0387 9536 0861.
If you would find it, please call at 313-666-7440 or write an email here: real.slim.shady@gmail.com."""

from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI

anonymizer = PresidioReversibleAnonymizer()

template = """Rewrite this text into an official, short email:

{anonymized_text}"""
prompt = PromptTemplate.from_template(template)
llm = ChatOpenAI(temperature=0)

chain = {"anonymized_text": anonymizer.anonymize} | prompt | llm
response = chain.invoke(text)
print(response.content)

Dear Sir/Madam,

We regret to inform you that Monique Turner has recently misplaced his wallet, which contains a sum of cash and his credit card with the number 213152056829866.

If you happen to come across this wallet, kindly contact us at (770)908-7734x2835 or send an email to barbara25@example.net.

Thank you for your cooperation.

Sincerely,
[Your Name]
```

##### 이제 시퀀스에 비익명화 단계를 추가하자

```python
chain = chain | (lambda ai_message: anonymizer.deanonymize(ai_message.content))
response = chain.invoke(text)
print(response)

Dear Sir/Madam,

We regret to inform you that Slim Shady has recently misplaced his wallet, which contains a sum of cash and his credit card with the number 4916 0387 9536 0861.

If you happen to come across this wallet, kindly contact us at 313-666-7440 or send an email to real.slim.shady@gmail.com.

Thank you for your cooperation.

Sincerely,
[Your Name]
```

##### 익명화된 데이터가 모델 자체에 부여되어 외부로 유출되지 않도록 보호되었다. 그런 다음 모델의 응답을 처리하고 사실 값을 실제 값으로 대체했다.

## 추가 내용
##### deanonymizer_mapping 매개변수는 원래 값에 대한 가짜 값의 매핑을 저장한다.

```python
from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer

anonymizer = PresidioReversibleAnonymizer(
    analyzed_fields=["PERSON", "PHONE_NUMBER", "EMAIL_ADDRESS", "CREDIT_CARD"],
    faker_seed=42,
)

anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com. "
    "By the way, my card number is: 4916 0387 9536 0861"
)

anonymizer.deanonymizer_mapping

{'PERSON': {'Maria Lynch': 'Slim Shady'},
 'PHONE_NUMBER': {'7344131647': '313-666-7440'},
 'EMAIL_ADDRESS': {'jamesmichael@example.com': 'real.slim.shady@gmail.com'},
 'CREDIT_CARD': {'4838637940262': '4916 0387 9536 0861'}}
```

##### 더 많은 텍스트를 익명화하면 새로운 매핑 항목이 생성된다.

```python
print(
    anonymizer.anonymize(
        "Do you have his VISA card number? Yep, it's 4001 9192 5753 7193. I'm John Doe by the way."
    )
)

anonymizer.deanonymizer_mapping

{'PERSON': {'Maria Lynch': 'Slim Shady', 'William Bowman': 'John Doe'},
 'PHONE_NUMBER': {'7344131647': '313-666-7440'},
 'EMAIL_ADDRESS': {'jamesmichael@example.com': 'real.slim.shady@gmail.com'},
 'CREDIT_CARD': {'4838637940262': '4916 0387 9536 0861',
  '3537672423884966': '4001 9192 5753 7193'}}
```

##### 내장 메모리 덕분에 이미 감지되어 익명화된 엔터티는 이후 처리되는 텍스트에서 동일한 형식을 취하므로 매핑에 중복 항목은 존재하지 않는다.

##### 나중에 사용할 수 있도록 매핑 자체를 파일에 저장할 수 있다.

```python
# save
anonymizer.save_deanonymizer_mapping("deanonymizer_mapping.json")

# load
anonymizer.load_deanonymizer_mapping("deanonymizer_mapping.json")
```

### 맞춤형 비익명화
##### 기본 비익명화는 텍스트의 하위 문자열을 매핑 항목과 정확하게 일치시키는 것이다.
##### 하지만, LLM의 비결정성으로 모델이 개인 데이터의 형식을 약간 변경하거나 오타를 만들 수 있다.
##### 따라서 적절한 프롬프트 엔지니어링을 고려하거나 교체 전략을 구현해보자.
##### deanonymizer_matching_strategies와 같은 퍼지 일치를 사용해보자

```python
from langchain_experimental.data_anonymizer.deanonymizer_matching_strategies import (
    case_insensitive_matching_strategy,
)

# 원본 이름: Maria Lynch
print(anonymizer.deanonymize("maria lynch"))
print(
    anonymizer.deanonymize(
        "maria lynch", deanonymizer_matching_strategy=case_insensitive_matching_strategy
    )
)

maria lynch
Slim Shady
```

```python
from langchain_experimental.data_anonymizer.deanonymizer_matching_strategies import (
    fuzzy_matching_strategy,
)

# 원본 이름: Maria Lynch
# 원본 전화번호: 7344131647
print(anonymizer.deanonymize("Call Maria K. Lynch at 734-413-1647"))
print(
    anonymizer.deanonymize(
        "Call Maria K. Lynch at 734-413-1647",
        deanonymizer_matching_strategy=fuzzy_matching_strategy,
    )
)

Call Maria K. Lynch at 734-413-1647
Call Slim Shady at 313-666-7440
```

##### 단일 방법보다 결합된 방법이 가장 잘 작동한다.



*   먼저 일치검색 전략을 적용하자
*   그 다음 퍼지 전략을 사용하여 나머지를 일치시키자

```python
from langchain_experimental.data_anonymizer.deanonymizer_matching_strategies import (
    combined_exact_fuzzy_matching_strategy,
)

# Changed some values for fuzzy match showcase:
# - "Maria Lynch" -> "Maria K. Lynch"
# - "7344131647" -> "734-413-1647"
# - "213186379402654" -> "2131 8637 9402 654"
print(
    anonymizer.deanonymize(
        (
            "Are you Maria F. Lynch? I found your card with number 4838 6379 40262.\n"
            "Is this your phone number: 734-413-1647?\n"
            "Is this your email address: wdavis@example.net"
        ),
        deanonymizer_matching_strategy=combined_exact_fuzzy_matching_strategy,
    )
)

Are you Slim Shady? I found your card with number 4916 0387 9536 0861.
Is this your phone number: 313-666-7440?
Is this your email address: wdavis@example.net
```
